### Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación

### Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones 2022
Búsqueda y Recomendación para Textos Legales

Mentor: Jorge E. Pérez Villella

# Práctico Introducción al Aprendizaje Automático

Integrantes:
* Fernando Agustin Cardellino
* Adrian Zelaya

### Objetivos:

Probar distintos modelos de clasificación para evaluar la performance y la exactitud de predicción de cada modelo. 

* Utilizando el corpus normalizado en el práctico anterior, transformar el texto en vectores numéricos utilizando scikit-learn comparando los 3 modelos de vectorización. Explicar cada uno estos modelos.

* Clasificar los documentos por fuero. Trabajaremos con los siguientes modelos de clasificación de la librería scikit-learn: Logistic Regresion, Naive Bayes y SVM. En cada modelo probar distintos hiperparámetros, generar la Matriz de Confusión y la Curva ROC. Explicar los resultados obtenidos.

* Determinar y justificar cual es el modelo con mejor performance.

* Predecir el fuero de un documento utilizando el mejor modelo.

Opcional:

* Opcional: Profundizar el tema de stop words y cómo generar uno propio.

Fecha de Entrega: 29 de julio de 2022



In [8]:
# Importamos las librerías necesarias

import numpy as np
import pandas as pd
import os
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator
from nltk.corpus import stopwords
from nltk.stem import porter, snowball
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
#import nltk
#nltk.download('stopwords')  # Descomentar para bajar las stopwords
# Ref: https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/

In [2]:
# Cargamos modelo de spacy y stopwords de NLTK
nlp = spacy.load("es_core_news_sm")

CURR_DIR = os.getcwd()  # Gets current directory
STOPWORDS_ES = stopwords.words('spanish')
BREAKPOINT=5  # None para analizar todos los documentos, sino un número para analizar hasta n documentos
MAX_WORDS=1000
IMG_NAME = "legal-icon-png"


In [3]:
def getListOfFiles(dirName, return_dir=False):
    # create a list of file and sub directories
    # names in the given directory
    files = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for file in files:
        # Create full path
        fullPath = os.path.join(dirName, file)
        # If entry is a directory then get the list of files in this directory
        if os.path.isdir(fullPath):
            if return_dir:
                allFiles.append(fullPath.split(os.sep)[-1])
            else:
                allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)

    return allFiles

In [4]:
def get_tokens(files_path, fuero_name=None, breakpoint=None, object_type=None):
    """
    Función que arma el corpus de palabras y las tokeniza, utilizando SpaCy.
    Si `breakpoint` != None, dar el número de docs con el que se interrumpe la función
    object_type: indica el tipo de objeto que devuelve, i.e. 'token'/None (SpaCy object `token`), 'word' (texto/string) 
    or 'entity' (SpaCy object `entity`) 
    """
    def get_conditions(token, case):
        if case == 'entity':
            return True
        else:
            return token.is_alpha and not token.is_stop
    
    corpus = {}

    tokens = []

    i = 0
    for filename in getListOfFiles(files_path):
        file_name = filename.split(os.sep)[-1]
        fuero = fuero_name if fuero_name is not None else filename.split(os.sep)[-2]
        if fuero not in corpus.keys():
            corpus[fuero] = {}

        with open(filename, encoding='utf-8') as file:
            file_text = file.read()
            # Tokenizamos el corpus
            nlp_doc = nlp(file_text)
            corpus[fuero][file_name] = nlp_doc
            
            iterable = nlp_doc.ents if object_type == 'entity' else nlp_doc
            
            for token in iterable:
                if get_conditions(token, case=object_type):
                    tokens.append(# lematizamos y pasamos a minúscula (en modo 'palabras'/'word' solamente)
                        token.lemma_.lower() if object_type == 'word' else token
                    )
                    
        # remover esta sección (testing)
        if breakpoint:
            i += 1
            if i > breakpoint:
                break
    return tokens, corpus 


def get_lemmas_stem_from_tokens(tokens, stemmer, language='spanish'):
    aux_dict = {
        'word': [token.lower_ for token in tokens],
        'lemma': [token.lemma_.lower() for token in tokens],
        'stem': [stemmer.stem(token.lower_) for token in tokens]
    }
    return pd.DataFrame(aux_dict)
    
    
def get_conteo_palabras(palabras):
    """Función que genera un pandas DataFrame con la frecuencia de las palabras
    """
    palabras_df = pd.DataFrame([{'palabra': str(x).lower()} for x in palabras])
    # print(corpus_df.head())

    return palabras_df.groupby(['palabra'])['palabra'].count().sort_values(ascending=False)

    
def generar_corpus(file_path, breakpoint, fuero_name=None, verbose=True, object_type='word'):
    if verbose:
        print("Generamos Corpus de palabras y conteo de frecuencias")
    
    # Generamos el corpus de palabras, y el diccionario con el mapeo de los fueros con sus respectivos documentos
    palabras, corpus_dic = get_tokens(file_path, fuero_name=fuero_name, breakpoint=breakpoint, object_type='word')

    frecuencia_palabras_df = get_conteo_palabras(palabras)
    
    if verbose:
        print(f"Algunas palabras: {palabras[:5]}")
    
    return frecuencia_palabras_df, palabras


In [5]:
# Ubicación de los documentos
filesDir = os.path.join(CURR_DIR, "Documentos")

# Obtenemos lista de los fueros
fueros = getListOfFiles(filesDir, return_dir=True)

In [10]:
n_docs = BREAKPOINT if not None else 'todos'
    
print(f"\nAnálisis para {n_docs} documentos\n")

frecuencia_palabras_df, corpus = generar_corpus(filesDir, BREAKPOINT, object_type='word', verbose=False)

print(frecuencia_palabras_df.head())

print(corpus)
    


Análisis para 5 documentos

palabra
fs      171
niño     91
auto     88
hijo     85
art      83
Name: palabra, dtype: int64
['córdoba', 'veinticuatro', 'septiembre', 'mil', 'dieciocho', 'vistos', 'auto', 'caratulado', 'control', 'legalidad', 'ley', 'art', 'expte', 'n', 'venido', 'juzgado', 'niñez', 'adolescencia', 'violencia', 'familiar', 'nominación', 'secretaría', 'ex', 'men', 'sec', 'cargo', 'carlos', 'maría', 'lópez', 'peña', 'resultar', 'i', 'fs', 'comparecer', 'señora', 'asesora', 'letrada', 'civil', 'quinto', 'turno', 'interponer', 'recurso', 'reposición', 'apelación', 'subsidio', 'proveído', 'fecha', 'fs', 'disponer', 'tener', 'consideración', 'precepto', 'rector', 'actuación', 'interés', 'superior', 'niño', 'constancia', 'obrant', 'fs', 'vta', 'surgir', 'señor', 'vivir', 'domicilio', 'sito', 'calle', 'b', 'barrio', 'quebrado', 'rosas', 'conteste', 'domicilio', 'fijado', 'registro', 'electoral', 'obra', 'constancia', 'fs', 'tornar', 'absolutamente', 'abstracto', 'esgrimido', '

In [17]:
print(len(corpus))

12314


Utilizando el corpus normalizado en el práctico anterior, transformar el texto en vectores numéricos utilizando scikit-learn comparando los 3 modelos de vectorización. Explicar cada uno estos modelos.

# CountVectorizer
(acá iría descripción de qué es)


In [15]:
# CountVectorizer
# Ver warning: https://stackoverflow.com/questions/57340142/user-warning-your-stop-words-may-be-inconsistent-with-your-preprocessing
vectorizer_1 = CountVectorizer(strip_accents='ascii', stop_words=STOPWORDS_ES) # Both ‘ascii’ and ‘unicode’ use NFKD normalization from unicodedata.normalize.
X_1 = vectorizer_1.fit_transform(corpus) # Learn the vocabulary dictionary and return document-term matrix.
print(vectorizer_1.get_feature_names_out())
print(X_1.toarray())
print(X_1.toarray().shape)


['ab' 'abandonar' 'abandono' ... 'xxxxxx' 'yerro' 'zavala']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(12314, 2692)


# TfidfVectorizer
(descripción)

In [16]:
# ver normas L1 o L2 (default: L2)
# ver de pasar los documentos y configurar los parámetros de preprocesamiento
vectorizer_2 = TfidfVectorizer(strip_accents='ascii', stop_words=STOPWORDS_ES)
X_2 = vectorizer_2.fit_transform(corpus)
print(vectorizer_2.get_feature_names_out())
print(X_2.toarray())
print(X_2.toarray().shape)

['ab' 'abandonar' 'abandono' ... 'xxxxxx' 'yerro' 'zavala']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(12314, 2692)


# HashingVectorizer

In [24]:
# además de norm, ver n_features, alternate_sign
# ver de pasar los documentos y configurar los parámetros de preprocesamiento
vectorizer_3 = HashingVectorizer(strip_accents='ascii', stop_words=STOPWORDS_ES)
X_3 = vectorizer_3.fit_transform(corpus)
print(X_3)
#print(vectorizer_3.get_feature_names_out())
#print(X_3.toarray())
#print(X_3.toarray().shape)

  (0, 675406)	1.0
  (1, 291381)	1.0
  (2, 650436)	-1.0
  (3, 855705)	1.0
  (4, 826043)	1.0
  (5, 488516)	1.0
  (6, 740200)	1.0
  (7, 509841)	-1.0
  (8, 187316)	-1.0
  (9, 67541)	-1.0
  (10, 905664)	1.0
  (11, 733020)	-1.0
  (12, 933792)	-1.0
  (14, 521219)	1.0
  (15, 879624)	-1.0
  (16, 748186)	1.0
  (17, 1020752)	1.0
  (18, 519346)	-1.0
  (19, 244029)	-1.0
  (20, 336234)	1.0
  (21, 949962)	-1.0
  (22, 802897)	1.0
  (23, 949760)	1.0
  (24, 236954)	-1.0
  (25, 689098)	1.0
  :	:
  (12289, 528089)	-1.0
  (12290, 303001)	1.0
  (12291, 812070)	1.0
  (12292, 898506)	-1.0
  (12293, 571067)	-1.0
  (12294, 441442)	-1.0
  (12295, 928531)	1.0
  (12296, 757673)	-1.0
  (12297, 121300)	1.0
  (12298, 622630)	-1.0
  (12299, 914213)	-1.0
  (12300, 251390)	-1.0
  (12301, 1023814)	1.0
  (12302, 539007)	-1.0
  (12303, 438152)	1.0
  (12304, 764682)	1.0
  (12305, 580091)	-1.0
  (12306, 951425)	-1.0
  (12307, 119241)	1.0
  (12308, 733020)	-1.0
  (12309, 65782)	-1.0
  (12310, 794531)	1.0
  (12311, 339853)	1.0